In [77]:
import awswrangler as awr
import pandas as pd

file_path = r"C:\Users\raphael.almeida\Documents\Processos\historico_placas\sql\all_boards_sets.sql"
with open (file_path, 'r') as r:
    query = r.read()

df_sets = awr.athena.read_sql_query(query, database='silver')

CONFERÊNCIAS

In [78]:
df_sets['status_conjunto'].unique()
# se ATIVO NOVO RENOVAÇÃO é placa vigente, porque tem placas que nunca mudam para ATIVO, ficam como estão
# ['CANCELADO', 'CANCELADA', 'FINALIZADO', 'FINALIZADA', 'NAO RENOVADO']

<StringArray>
[                 'ATIVO',             'FINALIZADO',              'CANCELADO',
            'DESISTÊNCIA',              'RENOVAÇÃO',                   'NOVO',
            'SUBSTITUÍDO', 'AGENDADO PARA ATIVAÇÃO', 'DEVOLVIDO PARA UNIDADE',
      'AGUARDANDO ATIVAR',  'AGUARDANDO ASSINATURA',           'NAO RENOVADO',
                 'NEGADO',                     <NA>]
Length: 14, dtype: string

In [79]:
df_sets1 = df_sets[df_sets['placa']=='ZZZ9999']
df_sets1.sort_values('data_registro', ascending=False)

,placa,chassi,id_placa,matricula,conjunto,data_registro,data_cancelamento_conjunto,data_ativacao_conjunto,status_conjunto,empresa,data_atualizacao
42278,ZZZ9999,528LKACKDXPPD244,186298,2149,2833,2024-09-03,None,2024-09-03,SUBSTITUÍDO,Viavante,2025-04-22


criando cancelamentos e ativações

In [80]:
df_ativacoes = df_sets[df_sets['data_cancelamento_conjunto'].isnull()]

df_ativacoes = df_ativacoes[df_ativacoes['status_conjunto'].isin(['ATIVO', 'NOVO', 'RENOVAÇÃO'])] #averiguar o novo e renovação

df_ativacoes = df_ativacoes.drop_duplicates()

df_ativacoes['data_status'] = df_ativacoes['data_ativacao_conjunto']

df_ativacoes['vigencia'] = 'ATIVO'

In [81]:
df_cancelamentos = df_sets[df_sets['data_cancelamento_conjunto'].notna()]

df_cancelamentos = df_cancelamentos.drop_duplicates()

df_cancelamentos['data_status'] = df_cancelamentos['data_cancelamento_conjunto']

df_cancelamentos['vigencia'] = 'CANCELADO'

juntando as duas e filtrando ontem

In [82]:
df_aggregate = pd.concat([df_ativacoes, df_cancelamentos])

#declarando variáveis
today = pd.Timestamp.today()
today_date = today.date()
yesterday = today - pd.Timedelta(days=1)
yesterday_date= yesterday.date()
friday = today - pd.Timedelta(days=3)
friday_date = friday.date()
df_aggregate['data_status'] = pd.to_datetime(df_aggregate['data_status'])

if today.weekday() != 0:
    df_aggregate_yesterday = df_aggregate[df_aggregate['data_status'].dt.date==yesterday_date]
else:
    df_aggregate_yesterday = df_aggregate[df_aggregate['data_status'].dt.date==friday_date] 

#df_aggregate.sort_values('data_ativacao_conjunto', ascending=False)
df_aggregate_yesterday.sort_values('placa', ascending=False)

df_aggregate_yesterday['status'] = pd.NA
df_aggregate_yesterday['migration_from'] = pd.NA

df_aggregate_yesterday

C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_3960\4070733584.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aggregate_yesterday['status'] = pd.NA
C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_3960\4070733584.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aggregate_yesterday['migration_from'] = pd.NA


,placa,chassi,id_placa,matricula,conjunto,data_registro,data_cancelamento_conjunto,data_ativacao_conjunto,status_conjunto,empresa,data_atualizacao,data_status,vigencia,status,migration_from
617,HQR9D69,9BM9584619B651840,25097,4089,14320,2025-05-02,None,2025-05-16,ATIVO,Stcoop,2025-05-17,2025-05-16,ATIVO,<NA>,<NA>
1478,NPG1I75,9ADB0902DEM378117,17069,20451,14345,2025-05-05,None,2025-05-16,ATIVO,Stcoop,2025-05-16,2025-05-16,ATIVO,<NA>,<NA>
1479,NPH2G75,9ADM0442DEM378120,17071,20451,14345,2025-05-05,None,2025-05-16,ATIVO,Stcoop,2025-05-16,2025-05-16,ATIVO,<NA>,<NA>
2222,RRM4A62,98PTTH430NB125911,24879,11789,14259,2025-04-24,None,2025-05-16,ATIVO,Stcoop,2025-05-17,2025-05-16,ATIVO,<NA>,<NA>
2688,BDN9G30,97T0BN653L2001202,16924,21413,14425,2025-05-13,None,2025-05-16,ATIVO,Stcoop,2025-05-17,2025-05-16,ATIVO,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57972,ILN3I38,9BVA4CMA74E700274,186018,759,2696,2024-08-30,2025-05-16,2024-08-30,CANCELADO,Viavante,2025-05-16,2025-05-16,CANCELADO,<NA>,<NA>
58553,IOR8A69,93ZS2MRH088801142,196613,2695,3622,2024-10-01,2025-05-16,2024-09-30,CANCELADO,Viavante,2025-05-16,2025-05-16,CANCELADO,<NA>,<NA>
58860,APK1B71,9BVAS02C08E737150,16756312,8402,15965,2025-05-12,2025-05-16,2025-05-13,CANCELADO,Viavante,2025-05-16,2025-05-16,CANCELADO,<NA>,<NA>
58976,AUH7B15,9ADG0712BCM339622,13931,8555,14509,2025-04-22,2025-05-16,2025-04-22,CANCELADO,Viavante,2025-05-16,2025-05-16,CANCELADO,<NA>,<NA>


gerando a base geral de ativos

In [83]:

df_conf = df_sets


tratando a base geral 

tratando a data

In [84]:


df_conf['data_ativacao_conjunto'] = pd.to_datetime(df_conf['data_ativacao_conjunto'])
df_conf['data_cancelamento_conjunto'] = pd.to_datetime(df_conf['data_cancelamento_conjunto'])

if today.weekday() != 0:
    df_conf = df_conf[
        (
            (df_conf['data_ativacao_conjunto'].dt.date!=yesterday_date) &
            (df_conf['data_ativacao_conjunto'].dt.date!=today_date)
        )|     
             
        (
            (df_conf['data_cancelamento_conjunto'].dt.date!=yesterday_date) &
            (df_conf['data_cancelamento_conjunto'].dt.date!=today_date)
        )
        ]
else:
    df_conf = df_conf[
        (df_conf['data_ativacao_conjunto'].dt.date<friday_date) | 
        (df_conf['data_cancelamento_conjunto'].dt.date<friday_date) 
    ]



colocando data cancel e ativ em uma só coluna

In [85]:
df_ativacoes_geral = df_conf[df_conf['data_cancelamento_conjunto'].isnull()]

df_ativacoes_geral = df_ativacoes_geral[df_ativacoes_geral['status_conjunto'].isin(['ATIVO', 'NOVO', 'RENOVAÇÃO'])] #averiguar o novo e renovação

df_ativacoes_geral['data_status'] = df_ativacoes_geral['data_ativacao_conjunto']

df_ativacoes_geral['vigencia'] = 'ATIVO'

######################################################################

df_cancelamentos_geral = df_conf[df_conf['data_cancelamento_conjunto'].notna()]

df_cancelamentos_geral['data_status'] = df_cancelamentos_geral['data_cancelamento_conjunto']

df_cancelamentos_geral['vigencia'] = 'CANCELADO'

######################## CONCATENANDO

df_conf = pd.concat([df_ativacoes_geral, df_cancelamentos_geral])

C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_3960\1336788432.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cancelamentos_geral['data_status'] = df_cancelamentos_geral['data_cancelamento_conjunto']
C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_3960\1336788432.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cancelamentos_geral['vigencia'] = 'CANCELADO'


In [86]:
df_conf

,placa,chassi,id_placa,matricula,conjunto,data_registro,data_cancelamento_conjunto,data_ativacao_conjunto,status_conjunto,empresa,data_atualizacao,data_status,vigencia
0,BCS2B26,9BVRG30D3KE862876,13050,28,12886,2024-12-20,NaT,2024-12-25,ATIVO,Stcoop,2024-12-26,2024-12-25,ATIVO
2,RRQ4D44,9ADB0902PPM520264,12947,35,13559,2025-02-11,NaT,2025-02-18,ATIVO,Stcoop,2025-03-21,2025-02-18,ATIVO
3,GGR5A16,9BVRG20C0KE861038,14711,40,12972,2025-01-02,NaT,2025-01-05,ATIVO,Stcoop,2025-01-05,2025-01-05,ATIVO
16,QCN9D43,9A9V1503LLMFW4025,13358,121,13670,2025-02-18,NaT,2025-02-28,ATIVO,Stcoop,2025-03-01,2025-02-28,ATIVO
17,PQT2C74,9BVRG40D7GE831300,23920,121,7903,2024-04-16,NaT,2024-04-17,RENOVAÇÃO,Stcoop,2025-04-18,2024-04-17,ATIVO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
243137,OOM9F33,9BSR6X400H3904677,104966,43250,118611,2022-11-21,2024-01-03,2022-12-20,FINALIZADO,Segtruck,2024-01-03,2024-01-03,CANCELADO
243138,MDL9I89,9BSR4X2A033546119,105308,43251,88466,2021-12-20,2022-06-03,2021-12-20,CANCELADO,Segtruck,2022-06-03,2022-06-03,CANCELADO
243139,OFI9G15,9531M32P7CR249136,106527,43252,88471,2021-12-20,2023-01-05,2021-12-20,FINALIZADO,Segtruck,2022-04-22,2023-01-05,CANCELADO
243140,BAP9C41,9ADG1243EFM392220,58299,43253,120178,2022-12-21,2023-03-28,2022-12-23,CANCELADO,Segtruck,2023-03-28,2023-03-28,CANCELADO


criando as condições do status baeado no histórico montado

In [87]:
# Agrupar dados de conferência por chassi e benefício
df_conf_grouped = df_conf.groupby(['chassi']).agg({
    'empresa': list,
    'vigencia': list,
    'data_status': list
}).reset_index()

# Criar um dicionário para lookup rápido
conf_dict = df_conf_grouped.set_index(['chassi']).to_dict('index')

# Processar cada chassi/benefício único (validação)
for chassi in df_aggregate_yesterday['chassi'].drop_duplicates().values:
    if chassi in conf_dict:
        conf_data = conf_dict[chassi]
        
        # Ordenar datas e status
        dates = sorted([d for d in conf_data['data_status'] if pd.notna(d)], reverse=True)
        
        if dates:  # Se existe pelo menos uma data
            ultima_data = dates[0]
            idx_ultima = conf_data['data_status'].index(ultima_data)
            vigencia_ultimo = conf_data['vigencia'][idx_ultima]
            empresa_ultima = conf_data['empresa'][idx_ultima]
            
            # Atualizar registros correspondentes
            mask = (df_aggregate_yesterday['chassi'] == chassi)
            empresa_atual = df_aggregate_yesterday.loc[mask, 'empresa'].iloc[0]
            vigencia_atual = df_aggregate_yesterday.loc[mask, 'vigencia'].iloc[0]
            
            if vigencia_atual == 'ATIVO':
                if vigencia_ultimo == 'CANCELADO':
                    df_aggregate_yesterday.loc[mask, 'status'] = 'REATIVAÇÃO'
                elif empresa_ultima != empresa_atual:
                    df_aggregate_yesterday.loc[mask, 'status'] = 'MIGRAÇÃO'
                    df_aggregate_yesterday.loc[mask, 'migration_from'] = empresa_ultima
                else:
                    df_aggregate_yesterday.loc[mask, 'status'] = 'RENOVAÇÃO'
            else:
                df_aggregate_yesterday.loc[mask, 'status'] = 'CANCELADO'
    else:
        mask = (df_aggregate_yesterday['chassi'] == chassi)
        df_aggregate_yesterday.loc[mask, 'status'] = 'NOVO'

df = pd.concat([df_aggregate_yesterday, df_conf])
df.loc[df['vigencia'] == 'CANCELADO', 'status'] = 'CANCELADO'  #necessidade de forçar os com vigência CANCELADOS para status CANCELADO

df['data_registro'] = pd.to_datetime(df['data_registro'])
df['data_cancelamento_conjunto'] = pd.to_datetime(df['data_cancelamento_conjunto'])
df['data_ativacao_conjunto'] = pd.to_datetime(df['data_ativacao_conjunto'])
df['data_atualizacao'] = pd.to_datetime(df['data_atualizacao'])
df['data_status'] = pd.to_datetime(df['data_status'])

df['data_registro'] = df['data_registro'].dt.date
df['data_cancelamento_conjunto'] = df['data_cancelamento_conjunto'].dt.date
df['data_ativacao_conjunto'] = df['data_ativacao_conjunto'].dt.date
df['data_atualizacao'] = df['data_atualizacao'].dt.date
df['data_status'] = df['data_status'].dt.date
df['status'] = df['status'].fillna('NULL')
df['migration_from'] = df['migration_from'].fillna('NULL')

df = df.sort_values('data_status', ascending=False)
df['status'].value_counts()

status
NULL          110070
CANCELADO     104747
NOVO             136
REATIVAÇÃO        98
RENOVAÇÃO         22
MIGRAÇÃO          18
Name: count, dtype: int64

verificando se existem chassis que existam nos dois

In [88]:
# Verificando chassis existentes
chassis_yesterday = set(df_aggregate_yesterday['chassi'].unique())
chassis_all_active = set(df_conf['chassi'].unique())

# Encontrando chassis que existem em ambos
chassis_comuns = chassis_yesterday.intersection(chassis_all_active)

# Encontrando chassis que só existem em df_aggregate_yesterday
chassis_apenas_yesterday = chassis_yesterday - chassis_all_active

print(f"Total de chassis únicos em df_aggregate_yesterday: {len(chassis_yesterday)}")
print(f"Total de chassis únicos em df_conf: {len(chassis_all_active)}")
print(f"Total de chassis que existem em ambos: {len(chassis_comuns)}")
print(f"Total de chassis que só existem em df_aggregate_yesterday: {len(chassis_apenas_yesterday)}")

# Mostrando exemplos de chassis que só existem em df_aggregate_yesterday
if len(chassis_apenas_yesterday) > 0:
    print("\nExemplos de chassis que só existem em df_aggregate_yesterday:")
    for chassi in list(chassis_apenas_yesterday)[:5]:  # Mostrando os 5 primeiros
        print(f"Chassi: {chassi}")
        print(df_aggregate_yesterday[df_aggregate_yesterday['chassi'] == chassi][['placa', 'chassi', 'empresa', 'status_conjunto']].to_string())
        print()

Total de chassis únicos em df_aggregate_yesterday: 371
Total de chassis únicos em df_conf: 93175
Total de chassis que existem em ambos: 235
Total de chassis que só existem em df_aggregate_yesterday: 136

Exemplos de chassis que só existem em df_aggregate_yesterday:
Chassi: 9ADM0452HJM415627
         placa             chassi   empresa status_conjunto
45325  IYC3A98  9ADM0452HJM415627  Viavante           ATIVO

Chassi: 9BSR6X400R4069294
         placa             chassi   empresa status_conjunto
24202  SFL0I87  9BSR6X400R4069294  Viavante           ATIVO

Chassi: 98PTTH430RB150200
         placa             chassi   empresa status_conjunto
33700  RYX4F44  98PTTH430RB150200  Viavante            NOVO

Chassi: 9AA07133G1C032179
         placa             chassi   empresa status_conjunto
29198  MBK8E91  9AA07133G1C032179  Viavante           ATIVO

Chassi: 9BM958154NB274428
         placa             chassi   empresa status_conjunto
56555  RVH1H26  9BM958154NB274428  Viavante            NOVO


para o excel

In [89]:
save_path = r'C:\Users\raphael.almeida\Documents\Processos\historico_placas\historico_placas.xlsx'
df.to_excel(save_path, engine = 'openpyxl', index = False, sheet_name= 'historico_placas')